In [1]:
import pandas as pd
import ijson

In [63]:
#Retrieve data from json file
file = "dataset.json"
with open(file, 'r') as IO:
    objects = ijson.items(IO, 'meta.view.columns.item.fieldName')
    columnNames = list(objects)

In [77]:
#Get all column names
columnNames

[':sid',
 ':id',
 ':position',
 ':created_at',
 ':created_meta',
 ':updated_at',
 ':updated_meta',
 ':meta',
 'date_of_stop',
 'time_of_stop',
 'agency',
 'subagency',
 'description',
 'location',
 'latitude',
 'longitude',
 'accident',
 'belts',
 'personal_injury',
 'property_damage',
 'fatal',
 'commercial_license',
 'hazmat',
 'commercial_vehicle',
 'alcohol',
 'work_zone',
 'state',
 'vehicle_type',
 'year',
 'make',
 'model',
 'color',
 'violation_type',
 'charge',
 'article',
 'contributed_to_accident',
 'race',
 'gender',
 'driver_city',
 'driver_state',
 'dl_state',
 'arrest_type',
 'geolocation']

In [81]:
#Extract clean column names from columnNames
clean_names = [name for name in columnNames if (name[0] not in ":")]
clean_names

['date_of_stop',
 'time_of_stop',
 'agency',
 'subagency',
 'description',
 'location',
 'latitude',
 'longitude',
 'accident',
 'belts',
 'personal_injury',
 'property_damage',
 'fatal',
 'commercial_license',
 'hazmat',
 'commercial_vehicle',
 'alcohol',
 'work_zone',
 'state',
 'vehicle_type',
 'year',
 'make',
 'model',
 'color',
 'violation_type',
 'charge',
 'article',
 'contributed_to_accident',
 'race',
 'gender',
 'driver_city',
 'driver_state',
 'dl_state',
 'arrest_type',
 'geolocation']

In [101]:
with open(file, 'r') as IO:
    objects = ijson.items(IO, 'data.item')
    info = list(objects)

In [103]:
info[0]

[2118167,
 'EE8BC302-660F-48C4-B422-17427ECE821F',
 2118167,
 1482239054,
 '498050',
 1482239054,
 '498050',
 None,
 '2013-09-24T00:00:00',
 '17:11:00',
 'MCP',
 '3rd district, Silver Spring',
 'DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGISTRATION',
 '8804 FLOWER AVE',
 None,
 None,
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'MD',
 '02 - Automobile',
 '2008',
 'FORD',
 '4S',
 'BLACK',
 'Citation',
 '13-401(h)',
 'Transportation Article',
 'No',
 'BLACK',
 'M',
 'TAKOMA PARK',
 'MD',
 'MD',
 'A - Marked Patrol',
 [None, None, None, None, None]]

In [104]:
dataframe = pd.DataFrame(data, columns=clean_names)

In [107]:
dataframe.shape

(842, 35)

In [110]:
dataframe.date_of_stop = pd.to_datetime(dataframe.date_of_stop)

In [115]:
#Server connection with MongoClient
from pymongo import MongoClient
import json
server_conn = MongoClient(host='localhost', port=27017)
try:
    server_conn.admin.command('ismaster')
    print("Connected.")
except ConnectionFailure:
    print("Server not available")

Connected.


In [116]:
db = server_conn['traffic']
collections = db.collections
print(collections)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'traffic'), 'collections')


In [118]:
collections.insert_many(dataframe.to_dict('records'))

In [126]:
queryColor = collections.aggregate(
    [{
        "$group" : 
        {
            "_id":"$color", "count":
            {
                "$sum":1
            }
        }
    }])

In [127]:
for cursor in queryColor:
    print(cursor)

{'count': 1, '_id': 'BRONZE'}
{'count': 2, '_id': 'BROWN'}
{'count': 3, '_id': 'GREEN, LGT'}
{'count': 13, '_id': 'MAROON'}
{'count': 5, '_id': 'PURPLE'}
{'count': 29, '_id': 'GOLD'}
{'count': 3, '_id': 'CREAM'}
{'count': 33, '_id': 'TAN'}
{'count': 13, '_id': 'GREEN, DK'}
{'count': 20, '_id': 'BLUE, LIGHT'}
{'count': 43, '_id': 'GREEN'}
{'count': 4, '_id': 'ORANGE'}
{'count': 8, '_id': 'N/A'}
{'count': 111, '_id': 'WHITE'}
{'count': 67, '_id': 'BLUE'}
{'count': 145, '_id': 'SILVER'}
{'count': 21, '_id': 'BLUE, DARK'}
{'count': 73, '_id': 'RED'}
{'count': 76, '_id': 'GRAY'}
{'count': 1, '_id': None}
{'count': 159, '_id': 'BLACK'}
{'count': 12, '_id': 'BEIGE'}


In [128]:
queryArrest = collections.aggregate(
    [{
        "$group" : 
        {"_id":"$arrest_type", "count":
         {
             "$sum":1
         }
        }
    }])

In [129]:
for cursor in queryArrest:
    print(cursor)

{'count': 2, '_id': 'E - Marked Stationary Radar'}
{'count': 2, '_id': 'S - License Plate Recognition'}
{'count': 1, '_id': 'R - Unmarked Laser'}
{'count': 1, '_id': 'L - Motorcycle'}
{'count': 4, '_id': 'M - Marked (Off-Duty)'}
{'count': 35, '_id': 'Q - Marked Laser'}
{'count': 13, '_id': 'B - Unmarked Patrol'}
{'count': 3, '_id': 'O - Foot Patrol'}
{'count': 781, '_id': 'A - Marked Patrol'}


In [131]:
queryDay=collections.aggregate(
   [
     {
       "$project":
         {
           "dayOfWeek": { "$dayOfWeek": "$date_of_stop" },
         }
     }
   ])

In [133]:
data_chunk = pd.DataFrame(list(collections.find()))
data_chunk.head

<bound method NDFrame.head of                           _id accident agency alcohol        arrest_type  \
0    591593318dcfd926b4866d6e       No    MCP      No  A - Marked Patrol   
1    591593318dcfd926b4866d6f       No    MCP      No  A - Marked Patrol   
2    591593318dcfd926b4866d70       No    MCP      No  A - Marked Patrol   
3    591593318dcfd926b4866d71       No    MCP      No  A - Marked Patrol   
4    591593318dcfd926b4866d72       No    MCP      No  A - Marked Patrol   
5    591593318dcfd926b4866d73       No    MCP      No  A - Marked Patrol   
6    591593318dcfd926b4866d74       No    MCP      No  A - Marked Patrol   
7    591593318dcfd926b4866d75       No    MCP      No  A - Marked Patrol   
8    591593318dcfd926b4866d76       No    MCP      No  A - Marked Patrol   
9    591593318dcfd926b4866d77       No    MCP      No  A - Marked Patrol   
10   591593318dcfd926b4866d78       No    MCP      No  A - Marked Patrol   
11   591593318dcfd926b4866d79       No    MCP      No  A -

In [134]:
#Close Server Connection
server_conn.close()

In [137]:
print(data_chunk.date_of_stop.dt.dayofweek.value_counts())

4    133
0    130
1    129
3    128
6    113
5    113
2     96
Name: date_of_stop, dtype: int64
